In [7]:
import tensorflow as tf
import keras
from keras.models import Sequential
import tensorflow as tf  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2 
from PIL import Image
import keras
import glob
from tensorflow.keras.layers import Conv2D
from  tensorflow.nn import max_pool2d
from tensorflow.nn import max_pool2d
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.layers import BatchNormalization
from keras.layers import Flatten,Dense
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from PIL import Image
import cv2
from sklearn.preprocessing import LabelEncoder


In [9]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"tusharsachan","key":"a2a24c79c7dc7701349e905ab879f214"}'}

In [10]:
! pip install kaggle

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle(1).json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: ` chmod 600 ~/.kaggle/kaggle(1).json'


In [22]:
! kaggle datasets list


ref                                                       title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ruchi798/data-science-job-salaries                        Data Science Job Salaries                            7KB  2022-06-15 08:59:12           6715        226  1.0              
surajjha101/bigbasket-entire-product-list-28k-datapoints  BigBasket Entire Product List (~28K datapoints)      6MB  2022-06-22 12:51:18           2223         97  1.0              
victorsoeiro/netflix-tv-shows-and-movies                  Netflix TV Shows and Movies                          2MB  2022-05-15 00:01:23          16987        492  1.0              
sameepvani/nasa-nearest-earth-objects                     NASA - Nearest Earth Objects         

In [11]:
! kaggle datasets download -d l3llff/flowers

flowers.zip: Skipping, found more recently modified local copy (use --force to force download)


In [12]:
! unzip flowers.zip


Archive:  flowers.zip
replace flowers/astilbe/10091895024_a2ea04cda6_c.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [13]:
train_files=glob.glob("/content/flowers/*/**")
len(train_files)

15740

In [14]:
from random import shuffle 
shuffle(train_files)       
np.array(Image.open(train_files[0])).shape
(256, 256, 3)
X_train = np.zeros((len(train_files),28,28))
y_train = np.zeros(((len(train_files),1)))

In [15]:
key1 = 's/'
key2='/'

X_train=np.zeros((len(train_files),256,256),dtype=np.int)
Y_train = []
i=0
for fi in train_files:
    im=cv2.imread(fi, cv2.IMREAD_GRAYSCALE)
    new_array=cv2.resize(im,(256,256))
    X_train[i,:,:]=new_array
    label=train_files[i].partition(key1)[2].partition(key2)[0]
    Y_train.append(label)
    i=i+1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


In [18]:
Y_train = LabelEncoder().fit_transform(Y_train)
from tensorflow.keras.utils import to_categorical
Y_train_new = to_categorical(Y_train)
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=512, step=16),#min value of filter you want
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),#picks like mcq
        activation='relu',
        input_shape=(256,256,1)
    ),
      #for i in range(0,hp.Int(....)) this will also work
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=256, step=16),# ihave added this line extra in comparison to the old model
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(16, activation='softmax')
  ])
  
  model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model
!pip install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="AllFlowers")
tuner_search.search(X_train,Y_train_new,epochs=3,validation_split=0.1)

INFO:tensorflow:Reloading Oracle from existing project output/AllFlowers/oracle.json

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
240               |?                 |conv_1_filter
5                 |?                 |conv_1_kernel
160               |?                 |conv_2_filter
5                 |?                 |conv_2_kernel
48                |?                 |dense_1_units
0.001             |?                 |learning_rate



In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]
model.summary()
model.fit(X_train, Y_train, epochs=10, validation_split=0.1, initial_epoch=3)